# Housing Analysis for Phoenix, Arizona 

##### Warren Denetsosie, Scott Cearley, Munqiz Minhas


In [ ]:
# import libraries 
import pandas as pd 
import hvplot.pandas 
from pathlib import Path 
import datetime as dt
import altair as alt 
import requests
import json 

## Summary 

#### - Analyzing the Phoenix Housing Market pre-pandemic & post-pandemic to see how it affected total housing supply, prices, property types and the rental market. 

# Data Techniques 

#### - Redfin Data Center -- https://www.redfin.com/news/data-center/
#### - Redfin RapidAPI 
#### - Zillow API 

In [ ]:
# pulling API data from RedFin
url = "https://unofficial-redfin.p.rapidapi.com/properties/list"

querystring = {"region_id":"14240","region_type":"6","uipt":"1,2,3,7,8","status":"9","sf":"1,2,5,6,7","num_homes":"300","sold_within_days":"30"}

headers = {
	"X-RapidAPI-Key": "b6a21fcaa0msh2e9de61afa99433p14003ajsnd33bf08e42ad",
	"X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

dataPHX = response.json()

# display API pull status
print(f"Start API acquisition")
print(f"------------&&&&-------------------")

# print(json.dumps(dataPHX, indent=4))

querystring = {"region_id":"7102","region_type":"6","uipt":"1,2,3,7,8","status":"9","sf":"1,2,5,6,7","num_homes":"300","sold_within_days":"30"}

headers = {
	"X-RapidAPI-Key": "b6a21fcaa0msh2e9de61afa99433p14003ajsnd33bf08e42ad",
	"X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

dataGLEN = response.json()

# print(json.dumps(dataGLEN, indent=4))

In [ ]:
# cleaning data from RedFin 
property_id = []
property_type = []
beds = []
baths = []
square_ft_info = []
price_info = []
latitude = [] 
longitude = [] 

for homeData in dataGLEN['homes']:
    property_id.append(homeData['homeData']['propertyId'])
    property_type.append(homeData['homeData']['propertyType'])
    #beds.append(homeData['homeData']['beds']['value'])
    #baths.append(homeData['homeData']['baths']['value'])
    try:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']['value'])
    except KeyError:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']["NA"])
    price_info.append(homeData['homeData']['priceInfo']['amount']['value'])
    latitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['latitude'])
    longitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['longitude'])

for homeData in dataPHX['homes']:
    property_id.append(homeData['homeData']['propertyId'])
    property_type.append(homeData['homeData']['propertyType'])
    #beds.append(homeData['homeData']['beds']['value'])
    #baths.append(homeData['homeData']['baths']['value'])
    try:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']['value'])
    except KeyError:
        square_ft_info.append("0")
    price_info.append(homeData['homeData']['priceInfo']['amount']['value'])
    latitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['latitude'])
    longitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['longitude'])

In [ ]:
# creating dataframe 
data_dict = {"Property_ID": property_id, "Property_Type": property_type, "Square Footage": square_ft_info, "Price Info": price_info, "Latitude": latitude, "Longitude": longitude}
data_df = pd.DataFrame(data_dict)
data_df['Square Footage']=data_df['Square Footage'].astype(float)
data_df['Price Info']=data_df['Price Info'].astype(float)
data_df['PPSF'] = data_df['Price Info'] / data_df['Square Footage']
data_df

In [ ]:
# Housing Property Type Listings plot (600 homes)
data_df.hvplot.points(
    'Longitude',
    'Latitude', 
    geo = True, 
    xlabel = 'Latitude',
    ylabel = 'Longitude', 
    frame_width = 1000, 
    frame_height = 650, 
    tiles = 'OSM',
    color = 'Property_Type',
    size = 'PPSF',
    hover_cols = 'Price Info',
    title = 'Phoenix and Glendale Housing Property Type Listings'
)

In [ ]:
# import the phoenix housing listings data
phx_housing_listings_df = pd.read_csv(
                                    Path('Resources/Phoenix_listings.csv'), 
                                    index_col = 'rawAddress')

phx_housing_listings_df.drop(['Unnamed: 0', 'county', 'bedrooms', 'bathrooms', 'yearBuilt', 'lotSize',
                            'addressLine1', 'city', 'state', 'formattedAddress', 'lastSeen', 'listedDate', 'status', 
                            'removedDate', 'daysOnMarket', 'createdDate', 'id', 'addressLine2' ], 
                            axis=1, inplace=True)

phx_housing_listings_df = phx_housing_listings_df.dropna()

# review the first and last five rows of the dataframe 
display(phx_housing_listings_df.head())

In [ ]:
# plot to analyze housing listings info 
phx_housing_listings_df.hvplot.points(
    'longitude',
    'latitude', 
    xlabel = 'Latitude',
    ylabel = 'Longitude',
    geo = True, 
    frame_width = 1000, 
    frame_height = 650, 
    hover_cols = 'rawAddress',
    #colorbar = False,
    tiles = 'OSM',
    color = 'price',
    size = 'squareFootage',
    title = 'Phoenix Housing Property Listings -- Interactive Heat Map'
).opts(
    yformatter = '%.0f',
)

In [ ]:
# import the phoenix rental housing listings data
phx_housing_rental_listings_df = pd.read_csv(
                                    Path('Resources/Phoenix_rent_listings.csv'), 
                                    index_col = 'rawAddress')

phx_housing_rental_listings_df.drop(['Unnamed', 'addressLine1', 'formattedAddress', 'city', 'state', 'zipCode', 'county', 
                                    'bedrooms', 'bathrooms', 'createdDate', 'lastSeen', 'listedDate', 'status', 
                                    'removedDate', 'daysOnMarket', 'id', 'addressLine2', 'yearBuilt'], 
                                    axis=1, inplace=True)

phx_housing_rental_listings_df = phx_housing_rental_listings_df.dropna()

# review the first and last five rows of the dataframe 
display(phx_housing_rental_listings_df.head())

In [ ]:
# plot to analyze rental housing listings info 
phx_housing_rental_listings_df.hvplot.points(
    'longitude',
    'latitude', 
    xlabel = 'Latitude',
    ylabel = 'Longitude',
    geo = True, 
    frame_width = 1000, 
    frame_height = 650, 
    hover_cols = 'rawAddress',
    tiles = 'OSM',
    color = 'price',
    size = 'squareFootage',
    title = 'Phoenix Rental Housing Property Listings -- Interactive Heat Map'
)

In [ ]:
# import the housing units data from 2019 - 2022
phx_housing_units_df = pd.read_csv(Path('Resources\housing_inventory.csv'))

# review the first and last five rows of the dataframe 
display(phx_housing_units_df.head())
display(phx_housing_units_df.tail())

In [ ]:
# visual aggregation exploring the housing units by year in Phoenix
phx_housing_units_df.hvplot.bar(
    x = 'Months', 
    xlabel = 'Period', 
    ylabel = 'All Homes for Sale', 
    title = 'All Homes for Sale in Phoenix, Arizona from 2019 to 2022', 
    yformatter = '%.0f',
    rot = 90, 
    ylim = (1000,9000),
    height = 500, 
    width = 1250
).opts(
    color = 'blue',
    hover_color = 'orange'
)

In [ ]:
# read in the homes sold by year data
homes_sold_data = Path("Resources/homes_sold.csv")
homes_sold_df = pd.read_csv(homes_sold_data)

# drop extra columns
homes_sold_df.drop(['Day of Year', 'Year of Period End', 'Period Begin', 'adjusted_average_homes_sold_yoy', 'Average Homes Sold Yoy (tooltip)'], 
                                    axis=1, inplace=True)

homes_sold_df.rename(columns = {'Period End':'Period_End'}, inplace = True)

# sort dates
homes_sold_df['Period End'] = pd.to_datetime(homes_sold_df.Period_End)
homes_sold_df.sort_values(['Period End'], ascending = False)
homes_sold_df.drop(['Period_End'], axis = 1, inplace = True)

display(homes_sold_df.head())
display(homes_sold_df.tail())

In [ ]:
# clean the data
homes_sold_df['adjusted_average_homes_sold']=homes_sold_df['adjusted_average_homes_sold'].str.replace(',','').astype(float)

In [ ]:
homes_sold_df.dtypes

In [ ]:
# homes sold visual 
homes_sold_plot = homes_sold_df.hvplot(
    x="Period End", 
    xlabel = "Period End", 
    ylabel = "Adjusted Average Homes Sold",
    title = "Adjusted Average Homes Sold by Year",
    rot = 90,
    width = 1200, 
    height = 500, 
    )
homes_sold_plot

In [ ]:
# pull price per square foot csv 
ppsf_data = Path("Resources/2019_2022_ppsf.csv")
ppsf_df = pd.read_csv(ppsf_data)
ppsf_df.head()

In [ ]:
# transform table, reset index, and drop row [0]
ppsf_df = ppsf_df.T
ppsf_df.reset_index(drop=True, inplace=True)
ppsf_df.drop(ppsf_df.index[0], inplace=True)
ppsf_df.head()

In [ ]:
# data cleaning 
ppsf_df[1]=ppsf_df[1].astype(float)

ppsf_df.rename(columns = {
    0 : "Period",
    1 : "PPSF",
}, inplace = True)

display(ppsf_df.head())

In [ ]:
# PPSF visual 
ppsf_df.hvplot(
            x='Period',
            y='PPSF',
            xlabel = 'Period',
            rot=90,
            width = 1000, 
            height = 500, 
            title='Price per Square Foot by Year - Maricopa County', 
            ylabel='Price per Square Foot',
            grid=True)

In [ ]:
# percent change of PPSF visual 
ppsf_df['PPSF'] = ppsf_df['PPSF'].pct_change()
ppsf_df.hvplot(
            x='Period',
            y='PPSF', 
            xlabel = 'Period', 
            rot=90,
            width = 1000, 
            height = 500, 
            grid=True, 
            title='Percent Change in PPSF over Time - Maricopa County', 
            ylabel='Percent Change of PPSF')

In [ ]:
# pulling data from investor purchases csv and creating dataframe
investor_purchases_data = Path("Resources/investor_purchases_market_share.csv")
investor_purchases_df = pd.read_csv(investor_purchases_data)
investor_purchases_df

# removing commas and changing datatype to 'float'
investor_purchases_df['All Home Sales']=investor_purchases_df['All Home Sales'].str.replace(',','').astype(float)
investor_purchases_df['Investor Purchases']=investor_purchases_df['Investor Purchases'].str.replace(',','').astype(float)

# setting 'Quarter' column to datetime format
investor_purchases_df["Quarter"] = pd.to_datetime(investor_purchases_df["Quarter"])

display(investor_purchases_df.head())
display(investor_purchases_df.tail())

# Altair Data Visualization Package 

#### Altair offers a powerful and concise visualization grammar that enables you to build a wide range of statistical visualizations quickly

In [ ]:
# plotting altair bar chart utilizing transform_fold feature (overlaying and creating legend)
alt.Chart(investor_purchases_df).mark_bar().transform_fold(
    ['All Home Sales', 'Investor Purchases'], 
    as_=['variable', 'value']
).encode(
    x='Quarter:T',
    y='max(value):Q',
    color='variable:N'
).properties(
    title='Total Sales and Investor Purchases by Year',
    width=1200,
    height=700)

# Q&A 

# Links/Sources
###### - https://github.com/Mun-Min/Project_One
###### - https://www.redfin.com/news/data-center/ 
###### - https://rapidapi.com/apidojo/api/unofficial-redfin/ 
###### - https://rapidapi.com/realtymole/api/realty-mole-property-api/
###### - https://rapidapi.com/apimaker/api/zillow-com1/
###### - https://www.geeksforgeeks.org/top-8-python-libraries-for-data-visualization/
###### - https://altair-viz.github.io/getting_started/overview.html

###### - https://rise.readthedocs.io/en/stable/usage.html